In [1]:
#### This notebook takes original ProppLearner annotation files (StoryWorkbench format) and converts them into the dictionary structure used by my model

In [1]:
import sys
sys.path.append('../src')

import xmltodict 
from misc import save_dict, open_dict

In [4]:
rawAnnFileLocation = "ProppLearner/corefs_gold_xml/"

In [5]:
### For each story, go through Story Workbench tokens vs AllenNLP tokens, and create a dictionary mapping one type to the other

for storyNum in range(1, 16):

    # get basic token dict for story workbench
    fileName = "story" + str(storyNum) + ".sty"

    with open(rawAnnFileLocation + fileName, 'r', encoding='utf-8') as file:
        xml = file.read()

    dic = xmltodict.parse(xml)

    tokenDictSW = {}
    for i in range(len(dic['story']['rep'])):
        if dic['story']['rep'][i]['@id'] == "edu.mit.parsing.token":
            indexTokens = i
            break

    for token in dic['story']['rep'][indexTokens]['desc']:
        tokenDictSW[int(token['@id'])] = {'word':token['#text']}


    # get basic token dict for allennlp
    pFileName = "story" + str(storyNum) + ".p"
    corefs = open_dict("new_data/coref/" + pFileName)

    tokenDictA = {}
    for i, word in enumerate(corefs['tokenizedDocument']):
        tokenDictA[i] = {'word':word}

    # add allenNLP token indexes to the SW token dict
    keysA = list(tokenDictA.keys())
    keysSW = list(tokenDictSW.keys())

    repeat = True
    while repeat == True:
        repeat = False

        for index, (keyA,keySW) in enumerate(zip(keysA, keysSW)):
            
            # exact match
            if tokenDictA[keyA]['word'] == tokenDictSW[keySW]['word']:
                tokenDictSW[keySW]['Allen Index'] = keyA
                continue

            # difference in tokenization e.g. " vs ''
            lenKeysA = len(keysA)
            lenKeysSW = len(keysSW)

            # difference in tokenization e.g. " vs ''
            if index < lenKeysA - 1 and index < lenKeysSW - 1:
                if tokenDictSW[keysSW[index + 1]]['word'] == tokenDictA[keysA[index + 1]]['word']:
                    tokenDictSW[keySW]['Allen Index'] = keyA
                    keysA = keysA[index+1:]
                    keysSW = keysSW[index+1:]
                    repeat = True
                    
            if repeat == True:
                break

            # allenNLP has grouped stuff
            skip = 1
            repeatSkipLoop = True
            while skip < 6 and repeatSkipLoop:
                if index < lenKeysSW - skip - 1:
                    if tokenDictA[keysA[index + 1]]['word'] == tokenDictSW[keysSW[index + 1 + skip]]['word']:
                        
                        for i in range(skip + 1):
                            tokenDictSW[keysSW[index+i]]['Notes'] = 'AllenNLP has grouped tokens'
                            tokenDictSW[keysSW[index+i]]['Allen Index'] = keyA

                        keysA = keysA[index+1:]
                        keysSW = keysSW[index+skip+1:]
                        repeat = True
                        break
                    else:
                        skip += 1

                else:
                    repeatSkipLoop = False
                    break

            if repeat == True:
                break
            
            # Story workbench has grouped stuff
            skip = 1
            repeatSkipLoop = True
            while skip < 6 and repeatSkipLoop:
                if index < lenKeysA - skip - 1 and index < lenKeysSW - 1:
                    if tokenDictSW[keysSW[index + 1]]['word'] == tokenDictA[keysA[index + 1 + skip]]['word']:

                        tokenDictSW[keySW]['Allen Index'] = []
                        tokenDictSW[keySW]['Notes'] = "Story Workbench has grouped tokens"

                        for i in range(skip+1):
                            tokenDictSW[keySW]['Allen Index'].append(keysA[index + i])

                        keysSW = keysSW[index+1:]
                        keysA = keysA[index+skip+1:]
                        repeat = True
                        break
                    else:
                        skip += 1

                else:
                    repeatSkipLoop = False
                    print('Check end of dict manually. Story num:',storyNum)
                    break

            if repeat == True:
                break


    # save story workbench token dict:

    # save_dict(tokenDictSW, "ProppLearner/storyworkbench_allen_tokens_map/" + pFileName)
    # print("story",storyNum,"done")

FileNotFoundError: [Errno 2] No such file or directory: 'new_data/coref/story1.p'

In [ ]:
#### Now, use the dictionary created above to cinvert the StoryWorkbench coreference chains to my format, with AllenNLP indexes.

for storyNum in range(1,16):

    # get annotated dic file for story workbench
    fileName = "story" + str(storyNum) + ".sty"

    with open(rawAnnFileLocation + fileName, 'r', encoding='utf-8') as file:
        xml = file.read()

    dic = xmltodict.parse(xml)

    # get story workbench keys dict
    pFileName = "story" + str(storyNum) + ".p"
    tokenDictStoryWorkbench = open_dict("old_with_indexes_data/storyWorkbench_tokenDict/" + pFileName)

    # get allenNLP coref dict (for tokenized document field only)
    corefsA = open_dict("new_data/coref/" + pFileName)

    corefDict = {'clusters':[], 'tokenizedDocument':corefsA['tokenizedDocument']}


    # get coref chains from story WB annotations
    for i in range(len(dic['story']['rep'])):
        if dic['story']['rep'][i]['@id'] == "edu.mit.parsing.token":
            indexTokens = i
        elif dic['story']['rep'][i]['@id'] == "edu.mit.discourse.rep.coref":
            indexCoref = i
        elif dic['story']['rep'][i]['@id'] == "edu.mit.discourse.rep.refexp":
            indexRef = i

    keys = list(tokenDictStoryWorkbench.keys())

    for chainNum, chain in enumerate(dic['story']['rep'][indexCoref]['desc']):
        title = chain['#text'].split('|')[0]
        refs = chain['#text'].split('|')[1]

        corefDict['clusters'].append({'name': title})

        refs = refs.split(',')
        
        corefDict['clusters'][chainNum]['mentions'] = []

        for k, ref in enumerate(refs):

            # get position info from story workbench annotation file & allennlp Indexes
            for refExppression in dic['story']['rep'][indexRef]['desc']:
                
                if int(ref) == int(refExppression['@id']):
                    wordRefs = refExppression['#text']
                    break 

            if ',' in wordRefs:
                wordRefs = wordRefs.replace(",","~")
            
            wordRefsList = wordRefs.split('~')
            wordRefsList = [int(wordRefsList[0]), int(wordRefsList[-1])]

            wordRefsAllenNLP = [tokenDictStoryWorkbench[wordRefsList[0]]['Allen Index'] ,tokenDictStoryWorkbench[wordRefsList[1]]['Allen Index']  ]

            # get allenNLP indexes



            # get corresponding text from storybook workbench dictionary
            text = ""

            startIndex = keys.index(wordRefsList[0])
            endIndex = keys.index(wordRefsList[1])
            for i in range(endIndex - startIndex + 1):

                word = tokenDictStoryWorkbench[keys[startIndex + i]]['word']
                text += word + ' '
            
            text = text.strip()

            # add both of the above to dictionary, and append to list of mentions.
            corefDict['clusters'][chainNum]['mentions'].append({'position': wordRefsAllenNLP, 'position_StoryWorkbench':wordRefsList, 'text':text})

            # save the coref dict
            save_dict(corefDict, "ProppLearner/corefs_allen/" + pFileName)